# Tutorial 2: Power Analysis in Beignet

Learn statistical power analysis using Beignet's operators and TorchMetrics classes.

In [ ]:
import matplotlib.pyplot as plt
import torch

import beignet
from beignet.metrics import (
    CohensD,
    IndependentZTestPower,
    IndependentZTestSampleSize,
    ZTestPower,
)

torch.manual_seed(42)
print("Beignet Power Analysis Tutorial")

## 1. Effect Size Calculation

In [2]:
# Create sample data
control = torch.normal(0, 1, (30,))
treatment = torch.normal(0.5, 1, (30,))

# Calculate effect size
effect_size = beignet.cohens_d(control, treatment)
print(f"Cohen's d: {effect_size:.3f}")

Cohen's d: -0.504


## 2. Statistical Power Analysis

In [ ]:
# Power analysis
sample_size = torch.tensor(30.0)
power = beignet.z_test_power(effect_size, sample_size, alpha=0.05)
print(f"Statistical power: {power:.3f} ({power * 100:.1f}%)")

## 3. Sample Size Determination

In [ ]:
# Sample size calculation
required_n = beignet.z_test_sample_size(effect_size, power=0.8, alpha=0.05)
print(f"Required sample size for 80% power: {int(required_n)}")

## 4. TorchMetrics Integration

In [ ]:
# Initialize metrics
power_metric = ZTestPower(alpha=0.05, alternative="two-sided")
effect_metric = CohensD(pooled=True)

# Update with data
power_metric.update(effect_size, sample_size)
effect_metric.update(control, treatment)

# Compute results
computed_power = power_metric.compute()
computed_effect = effect_metric.compute()

print(f"Power metric result: {computed_power:.3f}")
print(f"Effect size metric result: {computed_effect:.3f}")

## 5. Interactive Plotting

In [ ]:
# Create power analysis plot
power_plotter = ZTestPower(alpha=0.05, alternative="two-sided")
fig = power_plotter.plot(
    dep_var="effect_size", sample_size=[10, 20, 30, 50], title="Power vs Effect Size"
)
plt.show()

print("Plot shows how power increases with effect size for different sample sizes")

## 6. PyTorch Lightning Integration

Example of training a model while monitoring statistical power:

In [ ]:
class PowerAnalysisCallback:
    def __init__(self, baseline_accuracy=0.75, target_power=0.8):
        self.baseline_accuracy = baseline_accuracy
        self.target_power = target_power
        self.power_metric = IndependentZTestPower(alpha=0.05, alternative="two-sided")
        self.sample_size_metric = IndependentZTestSampleSize(
            power=target_power, alpha=0.05, alternative="two-sided"
        )

    def on_validation_end(self, current_accuracy, current_sample_size, epoch):
        # Calculate effect size from accuracies
        accuracy_diff = current_accuracy - self.baseline_accuracy
        pooled_variance = self.baseline_accuracy * (1 - self.baseline_accuracy)
        effect_size = accuracy_diff / torch.sqrt(torch.tensor(pooled_variance))

        # Update metrics
        self.power_metric.update(effect_size, torch.tensor(float(current_sample_size)))
        self.sample_size_metric.update(effect_size)

        # Get recommendations
        current_power = self.power_metric.compute()
        recommended_size = self.sample_size_metric.compute()
        additional_needed = max(0, int(recommended_size) - current_sample_size)

        print(f"\nEpoch {epoch + 1} Power Analysis:")
        print(
            f"  Accuracy: {current_accuracy:.3f} vs baseline {self.baseline_accuracy:.3f}"
        )
        print(f"  Effect size: {effect_size:.3f}")
        print(f"  Current power: {current_power * 100:.1f}%")

        if additional_needed > 0:
            print(
                f"  📊 RECOMMENDATION: Collect {additional_needed} more samples to reach {self.target_power:.0%} power"
            )
        else:
            print("  ✅ SUFFICIENT: Current sample size achieves target power")

        # Reset for next epoch
        self.power_metric.reset()
        self.sample_size_metric.reset()

        return additional_needed


print("PowerAnalysisCallback defined - ready for ML training integration")

In [8]:
# Demo: Simulate model training with power monitoring
callback = PowerAnalysisCallback(baseline_accuracy=0.75, target_power=0.8)

# Simulate improving accuracy over epochs
accuracies = [0.76, 0.78, 0.82, 0.85, 0.87]
sample_size = 200

print("Simulating model training with power analysis:")
for epoch, acc in enumerate(accuracies):
    additional_needed = callback.on_validation_end(acc, sample_size, epoch)

print(
    "\nDemo complete! The callback provides real-time data collection recommendations."
)

Simulating model training with power analysis:

Epoch 1 Power Analysis:
  Accuracy: 0.760 vs baseline 0.750
  Effect size: 0.023
  Current power: 0.0%
  📊 RECOMMENDATION: Collect 46343 more samples to reach 80% power

Epoch 2 Power Analysis:
  Accuracy: 0.780 vs baseline 0.750
  Effect size: 0.069
  Current power: 0.0%
  📊 RECOMMENDATION: Collect 4972 more samples to reach 80% power

Epoch 3 Power Analysis:
  Accuracy: 0.820 vs baseline 0.750
  Effect size: 0.162
  Current power: 0.0%
  📊 RECOMMENDATION: Collect 750 more samples to reach 80% power

Epoch 4 Power Analysis:
  Accuracy: 0.850 vs baseline 0.750
  Effect size: 0.231
  Current power: 2.7%
  📊 RECOMMENDATION: Collect 266 more samples to reach 80% power

Epoch 5 Power Analysis:
  Accuracy: 0.870 vs baseline 0.750
  Effect size: 0.277
  Current power: 20.2%
  📊 RECOMMENDATION: Collect 124 more samples to reach 80% power

Demo complete! The callback provides real-time data collection recommendations.


## Summary

This tutorial covered:

1. **Effect size calculation** with `cohens_d`
2. **Power analysis** with `normal_power`
3. **Sample size planning** with `normal_sample_size`
4. **TorchMetrics integration** for stateful computation
5. **Interactive plotting** for visualization
6. **PyTorch Lightning integration** for ML workflows

### Key Benefits:
- **Real-time recommendations** for data collection
- **Statistical rigor** in ML model evaluation
- **Publication-ready visualizations**
- **Seamless PyTorch integration**

### Next Steps:
- Integrate `PowerAnalysisCallback` into your training loops
- Explore other power analysis functions (F-tests, independent samples)
- Use plotting features for research presentations
- Apply to your own datasets and experiments